In [1]:
from sqlalchemy import create_engine, MetaData, inspect, exc
from sqlalchemy import Table, Column, ForeignKey
from sqlalchemy import Float, Integer, String, DateTime

from config import user, password

In [2]:
MySQL_DB_Connection = f'{user}:{password}@localhost/moneyball_db'

try:
    engine = create_engine(f'mysql://{MySQL_DB_Connection}', echo=False)
    connection = engine.connect()
    print(f'\n<Connected> MySQL Datbase: moneyball_db')

except exc.OperationalError:
    MySQL_Base_Connection = f'{user}:{password}@localhost'
    engine = create_engine(f'mysql://{MySQL_Base_Connection}', echo=False)
    connection  = engine.connect()
    engine.execute('CREATE DATABASE IF NOT EXISTS moneyball_db;')
    engine = create_engine(f'mysql://{MySQL_DB_Connection}', echo=False)
    print(f'\n<Created> MySQL Database: moneyball_db [Connection:Successful]')


<Connected> MySQL Datbase: moneyball_db


In [3]:
Meta = MetaData(bind=engine)
Inspector = inspect(engine)

try:
    db_table_names = Inspector.get_table_names()[:]
    print(db_table_names)
except AttributeError:
    db_table_names = []

mstr_created = False
salary_created = False
moneyball_created = False

['mlb_players', 'moneyball']


In [4]:
# Create Required Tables
mstr_table_name = 'mlb_players'
if mstr_table_name not in db_table_names:
    mstr_created = True
    mstr_table = Table(mstr_table_name, Meta,
                       Column('playerID', String(50),ForeignKey('mlb_player_salary.c.playerID'), primary_key=True, nullable=False),
                       Column('nameFirst', String(50), nullable=False),
                       Column('nameLast', String(50), nullable=False),
                       Column('birthYear', DateTime),
                       Column('birthMonth', DateTime),
                       Column('birthDay', DateTime),
                       Column('birthCountry', String(50)),
                       Column('birthState', String(50)),
                       Column('birthCity', String(50)),
                       Column('height', Float),
                       Column('weight', Float),
                       Column('college', String(50)),
                       Column('bats', String(10)),
                       Column('throws', String(10)),
                       Column('debut', DateTime)
                      )

salary_table_name = 'mlb_player_salary'
if salary_table_name not in db_table_names:
    salary_created = True
    salary_table = Table('mlb_player_salary', Meta,
                         Column('s_id', Integer, primary_key=True),
                         Column('yearID', DateTime),
                         Column('teamID', String(50), ForeignKey('moneyball.c.Team'), nullable=False),
                         Column('lgID', String(50)),
                         Column('playerID', String(50), ForeignKey('mlb_players.c.playerID'), nullable=False),
                         Column('salary', Float)
                        )
                
moneyball_table_name = 'moneyball'
if moneyball_table_name not in db_table_names:
    moneyball_created = True
    moneyball_table = Table('moneyball', Meta,
                            Column('m_id', Integer, primary_key=True),
                            Column('Team', String(50), ForeignKey('mlb_player_salary.c.playerID'), nullable=False),
                            Column('Year', DateTime),
                            Column('RS', Float),
                            Column('RA', Float),
                            Column('W', Float),
                            Column('OBP', Float),
                            Column('SLG', Float),
                            Column('BA', Float),
                            Column('Playoffs', Float),
                            Column('RankSeason', Float),
                            Column('RankPlayoffs', Float),
                            Column('G', Float),
                            Column('OOBP', Float),
                            Column('OSLG', Float),                      
                           )    
Meta.create_all()
    
print(
    f'\n<Created:{mstr_created}> mstr_table >> Name: mlb_players\n'
    f'<Created:{salary_created}> salary_table >> Name: mlb_player_salary\n'
    f'<Created:{moneyball_created}> moneyball_table >> Name: moneyball'
     )

NoReferencedTableError: Foreign key associated with column 'mlb_player_salary.playerID' could not find table 'mlb_players.c' with which to generate a foreign key to target column 'playerID'